## Data Wrangling
---
The objective of this assignment is to clean and reshape a dataset, and calculate summary statistics on binge drinking prevalence and poverty rates in U.S. States.

## Data
---
Download the U.S. Chronic Disease Indicators (CDI) in .csv format and load it using pandas.read_csv(). Important: Make sure to exclude the .csv file from your GitHub repository by adding it to your .gitignore. If not excluded, the pushing to GitHub will fail.

In [2]:
# Import Python Library
import pandas as pd

In [3]:
# create variable data by reading csv file
# path "C:\Users\10499\Chau_Fung\hw02\U.S._Chronic_Disease_Indicators__CDI___2023_Release_20240922.csv"
data = pd.read_csv('U.S._Chronic_Disease_Indicators__CDI___2023_Release_20240922.csv', low_memory=False)
data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2014,2014,AR,Arkansas,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,5,AST,AST3_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN
1,2018,2018,CO,Colorado,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,8,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2018,2018,DC,District of Columbia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,11,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2017,2017,GA,Georgia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,13,AST,AST3_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN
4,2010,2010,MI,Michigan,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,26,AST,AST3_1,NMBR,RACE,HIS,NaN,NaN,NaN,NaN


## Data Selection and Reshaping
---
1. Select Relevant Columns <br>

We are only interested in a subset of variables. Select the following variables and remove all others:

* Binge Drinking:
  - Binge drinking prevalence among adults aged >= 18 years (overall, males, females), in crude prevalence percentages.
* Poverty:
  - Poverty prevalence, Crude Prevalence in Percent (overall only).

In [4]:
# Select relevant Columns whose "Question" variable contains both Binge Drinking and Poverty

# select requirement for Binge Drinking
gender_binge = (data['Stratification1'] == 'Overall') | (data['Stratification1'] == 'Male') | (data['Stratification1'] == 'Female')
condition_binge = ((data['Question'] == 'Binge drinking prevalence among adults aged >= 18 years') 
                   & (gender_binge)
                   & (data['DataValueType'] == 'Crude Prevalence'))

# select requirement for Poverty
gender_poverty = (data['Stratification1'] == 'Overall')
condition_poverty = ((data['Question'] == 'Poverty') 
                     & (gender_poverty) 
                     & (data['DataValueType'] == 'Crude Prevalence'))

# use both requirements above to select the relevant columns
r_columns = data[ (condition_poverty) | condition_binge]

2. Reshape the Data <br>

Convert the dataset from long format to wide format using appropriate pandas commands. Ensure you have the correct columns and rename them as follows:

* state: Name of the State
* stateabb: State Abbreviation
* year: Year of observation
* binge_all: Binge drinking prevalence (all adults)
* binge_male: Binge drinking prevalence (male adults)
* binge_female: Binge drinking prevalence (female adults)
* poverty: Poverty prevalence (overall population)

In [5]:
# Pivot the dataset first
pivot_data = r_columns.pivot_table(
    index=['LocationDesc', 'LocationAbbr', 'YearStart'], 
    columns=['Question', 'Stratification1'],
    values='DataValue',
    aggfunc='first'  # taking the first non-null value
)

# Since data is multi-index, we shall flatten the columns to make renaming easier
pivot_data.columns = ['_'.join(col).strip() for col in pivot_data.columns.values]

# Rename columns based on the new flattened names
pivot_data = pivot_data.rename(columns={
    'Binge drinking prevalence among adults aged >= 18 years_Overall': 'binge_all',
    'Binge drinking prevalence among adults aged >= 18 years_Male': 'binge_male',
    'Binge drinking prevalence among adults aged >= 18 years_Female': 'binge_female',
    'Poverty_Overall': 'poverty'
}, inplace=False)

# Reset index and rename index columns
pivot_data = pivot_data.reset_index(inplace=False)
pivot_data = pivot_data.rename(columns={
    'LocationDesc': 'state',
    'LocationAbbr': 'stateabb',
    'YearStart': 'year'
}, inplace=False)

In [6]:
# Important!!!!!
# Many number in DataVaule of the origin csv is actually object.
# To correctly show mean, sd, and min, etc., we need to convert objects into numbers

# Convert 'year' to integer
pivot_data['year'] = pd.to_numeric(pivot_data['year'], errors='coerce').astype(int)

# Convert 'binge_all' to float
pivot_data['binge_all'] = pd.to_numeric(pivot_data['binge_all'], errors='coerce')

# Convert 'binge_male' to float
pivot_data['binge_male'] = pd.to_numeric(pivot_data['binge_male'], errors='coerce')

# Convert 'binge_female' to float
pivot_data['binge_female'] = pd.to_numeric(pivot_data['binge_female'], errors='coerce')

# Convert 'poverty' to float
pivot_data['poverty'] = pd.to_numeric(pivot_data['poverty'], errors='coerce')

In [7]:

# Create a copy and rename it with the full names
full_named = pivot_data.copy()
full_named = full_named.rename(columns={
    'state': 'Name of the State',
    'stateabb': 'State Abbreviation',
    'year': 'Year of observation',
    'binge_all': 'Binge drinking prevalence (all adults)',
    'binge_male': 'Binge drinking prevalence (male adults)',
    'binge_female': 'Binge drinking prevalence (female adults)',
    'poverty': 'Poverty prevalence (overall population)'
}, inplace=False)

# Check for result
full_named.head(15)

,Name of the State,State Abbreviation,Year of observation,Binge drinking prevalence (female adults),Binge drinking prevalence (male adults),Binge drinking prevalence (all adults),Poverty prevalence (overall population)
0,Alabama,AL,2010,NaN,NaN,NaN,19.0
1,Alabama,AL,2011,7.9,20.2,13.7,19.0
2,Alabama,AL,2012,7.0,18.2,12.3,19.0
3,Alabama,AL,2013,6.5,16.3,11.2,18.7
4,Alabama,AL,2014,6.8,17.9,12.0,19.3
5,Alabama,AL,2015,7.5,16.2,11.6,18.5
6,Alabama,AL,2016,8.8,17.6,13.0,17.1
7,Alabama,AL,2017,8.0,17.4,12.4,16.9
8,Alabama,AL,2018,7.9,17.8,12.6,16.8
9,Alabama,AL,2019,8.9,16.4,12.4,15.5


3. Summary of the Cleaned Data <br>

Provide an overview of the cleaned dataset by printing:

* The shape of the data (shape)
* Summary statistics (describe)

In [8]:
# Show the shape of the data
print('Shape of the data:')
full_named.shape

Shape of the data:


(648, 7)

In [9]:
# Show the summary statistics of the data
print("Summary Statistics:")
full_named.describe(include='all')  # include='all' will show the summary of all columns, non-numeric ones included

Summary Statistics:


,Name of the State,State Abbreviation,Year of observation,Binge drinking prevalence (female adults),Binge drinking prevalence (male adults),Binge drinking prevalence (all adults),Poverty prevalence (overall population)
count,648,648,648.000000,594.000000,594.000000,594.000000,530.000000
unique,55,55,NaN,NaN,NaN,NaN,NaN
top,Alabama,AL,NaN,NaN,NaN,NaN,NaN
freq,12,12,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2015.509259,11.726599,22.064141,16.751515,14.718113
std,NaN,NaN,3.447989,2.781037,3.705719,3.066388,5.245498
min,NaN,NaN,2010.000000,5.000000,13.100000,9.600000,7.300000
25%,NaN,NaN,2013.000000,9.900000,19.625000,14.825000,11.600000
50%,NaN,NaN,2016.000000,11.700000,21.700000,16.600000,13.800000
75%,NaN,NaN,2018.250000,13.175000,24.100000,18.400000,16.600000


In [12]:
columns_to_describe = ['binge_female','binge_male','binge_all']

summary_stats = pivot_data[columns_to_describe].describe(include='all')

print(summary_stats)

       binge_female  binge_male   binge_all
count    594.000000  594.000000  594.000000
mean      11.726599   22.064141   16.751515
std        2.781037    3.705719    3.066388
min        5.000000   13.100000    9.600000
25%        9.900000   19.625000   14.825000
50%       11.700000   21.700000   16.600000
75%       13.175000   24.100000   18.400000
max       22.300000   33.800000   27.200000


4. Save Cleaned Data <br>

Save the final dataset as binge_clean.csv. This file should be submitted with your assignment.

In [25]:
# Save the DataFrame to a CSV file
full_named.to_csv('binge_clean.csv', index=False)  # index=False to not include row indices in the file

# note: binge_clean.csv will be saved in the current working directory. 
# If need it to be saved in a specific directory, specify the full path (e.g., C:/Users/10499/Chau_Fung/hw02).

## Data Transformation and Analysis
---
**1. Top 10 States for Binge Drinking** <br>
For the most recent year in the dataset, create a table showing the top 10 U.S. states with the highest overall binge drinking prevalence. Include binge drinking rates for males, females, and the overall population. Sort by the overall binge drinking rate.


In [26]:
# Find the most recent year
most_recent_year = pivot_data['year'].max()

# Select the dataset with ONLY the most recent year
recent_data = pivot_data[pivot_data['year'] == most_recent_year]

# Sort the data by overall binge drinking prevalence from high to low (descending order)
sorted_data = recent_data.sort_values(by='binge_all', ascending=False)

# Select the top 10 states
ten_binge_states = sorted_data.head(10)

# To set the index from 1 to 10 instead of 0 to 9
# Reset the index without keeping the old index
ten_binge_states = ten_binge_states.reset_index(drop=True)

# Adjust the index to start from 1
ten_binge_states.index = ten_binge_states.index + 1

# Omit variable column 'poverty' and Rename the columns to full names
state_full = ten_binge_states.copy().drop('poverty', axis = 1)
state_full = state_full.rename(columns={
    'state': 'Name of the State',
    'stateabb': 'State Abbreviation',
    'year': 'Year of observation',
    'binge_all': 'Binge drinking prevalence (all adults)',
    'binge_male': 'Binge drinking prevalence (male adults)',
    'binge_female': 'Binge drinking prevalence (female adults)',
    'poverty': 'Poverty prevalence (overall population)'
}, inplace=False)

# Check result
state_full

,Name of the State,State Abbreviation,Year of observation,Binge drinking prevalence (female adults),Binge drinking prevalence (male adults),Binge drinking prevalence (all adults)
1,District of Columbia,DC,2021,18.8,26.7,22.5
2,Wisconsin,WI,2021,17.9,25.9,21.9
3,North Dakota,ND,2021,15.0,26.3,20.8
4,Montana,MT,2021,15.4,25.6,20.5
5,Iowa,IA,2021,14.3,26.1,20.1
6,South Dakota,SD,2021,14.8,24.4,19.6
7,Nebraska,NE,2021,14.3,24.8,19.5
8,Colorado,CO,2021,14.2,23.2,18.7
9,Minnesota,MN,2021,13.9,21.9,17.9
10,Kansas,KS,2021,13.4,22.1,17.7


**2. Annual Growth Rates in Binge Drinking** <br>
Calculate the average annual growth rate of binge drinking prevalence (for the overall population) across states, over the available years.

One way to calculate growth rates:

* Use groupby to group by state.
* Use the first() and last() functions to find the first and last available data points for each state.
* Calculate the percentage change and divide it by the number of years in the dataset, or use pct_change() to simplify the process.
* Provide a table showing the 5 states with the largest increases and the 5 states with the largest decreases in binge drinking prevalence over the time period.

In [27]:
# Create a copy dataFrame that only has state, stateabb, year, and binge drinking prevalence (for the overall population)
df_binge_all = pivot_data.copy().drop(['binge_male','binge_female', 'poverty'] , axis = 1)
df_binge_all.head()

# Sorted data by 'year' so before applying first() and last() 
# Thus, we can ensure the data is chronological so that these functions pick the correct starting and ending points
df_binge_all = df_binge_all.sort_values('year')

# Use groupby to group by state
df_binge_all = df_binge_all.groupby('state')

# Use the first() and last() functions to find the first and last available data points for each state.
first_pt = df_binge_all.first()
last_pt = df_binge_all.last()

# Calculate the percentage change and divide it by the number of years in the dataset, or use pct_change() to simplify the process.
number_years = last_pt['year'] - first_pt['year']
percent_change = ((last_pt['binge_all'] - first_pt['binge_all']) / first_pt['binge_all']) * 100
annual_growth = percent_change / number_years


In [28]:
# Provide a table showing the 5 states with the largest increases and the 5 states with the largest decreases in binge drinking prevalence over the time period.

# First, create a dataframe to store result
growth_results = pd.DataFrame({
    'Annual growth (%)': annual_growth
})

# Ensure the index (state names) is a column for sorting purposes
growth_results.reset_index(inplace=False)

# Sort by 'annual_growth' to find the largest increases and decreases
# one way to do it
sorted_for_increases = growth_results.sort_values(by='Annual growth (%)', ascending=False)
top_5_increases = sorted_for_increases.head(5) 
# another way to do
top_5_decreases = growth_results.nsmallest(5, 'Annual growth (%)')

# Add a column of index to show ranking
top_5_increases['Index'] = range(1, 6)
top_5_decreases['Index'] = range(1, 6)


# To fit everything into one table as requested:
# Add a new column to each dataframe to indicate the change type
top_5_increases['Change'] = 'Increase'
top_5_decreases['Change'] = 'Decrease'

# Concatenate the two dataframes vertically
combined_table = pd.concat([top_5_increases, top_5_decreases], ignore_index=False)

# Display the combined table
display(combined_table)

C:\Users\10499\AppData\Local\Temp\ipykernel_7808\2214782531.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_increases['Index'] = range(1, 6)
C:\Users\10499\AppData\Local\Temp\ipykernel_7808\2214782531.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_increases['Change'] = 'Increase'


,Annual growth (%),Index,Change
state,,,
Tennessee,3.818182,1,Increase
West Virginia,1.350135,2,Increase
Louisiana,0.621118,3,Increase
Kansas,0.374332,4,Increase
North Carolina,0.059809,5,Increase
Virgin Islands,-3.421053,1,Decrease
Delaware,-3.313927,2,Decrease
Illinois,-3.241107,3,Decrease
Nevada,-2.883675,4,Decrease
